# Load Dependencies

In [1]:
import numpy as np
import pandas as pd; pd.options.display.max_columns = 200
import geopandas as gpd
import linref as lr

In [51]:
# Define some global variables
PROJECT_CRS = 'EPSG:3857'

# Load Data

In [52]:
# Point this to the location of the Geopackage file
fp = '/mnt/h/H0000/00CMH.00251.00_S_ODOT HSIP Safety Services/02C Python Training/Example Data/franklin_county_training_data.gpkg'

# List all the layers in the file
gpd.list_layers(fp)

,name,geometry_type
0,crashes,Point
1,roadways,MultiLineString


In [56]:
# Load roadway data
# Includes all arterial and collector roadways in Franklin County, excluding freeways and interstates
roadways = gpd.read_file(fp, layer='roadways')
roadways.to_crs(PROJECT_CRS, inplace=True)

# Load crash data
# Includes crashes from 2019-2023 in Franklin County of KABC severities, on the selected ODOT roadways
crashes = gpd.read_file(fp, layer='crashes')
crashes.to_crs(PROJECT_CRS, inplace=True)

print(f'Data loaded: {len(roadways):,.0f} roadways, {len(crashes):,.0f} crashes')

Data loaded: 5,110 roadways, 25,872 crashes


# Crash Data Cleanup

## Decoding and Binning

Data dictionary available here: https://www.dot.state.oh.us/Divisions/Planning/LocalPrograms/LTAP/Documents/Crash_Data_Dictionary.pdf?ID=1229/1000

# Roadway Data Cleanup

# Linear Referencing

In [59]:
# Create events collections for managing linearly referenced data
roadways_ec = lr.EventsCollection(roadways, keys=['NLF_ID'], beg='CTL_BEGIN_', end='CTL_END_NB', geom='geometry')
crashes_ec = lr.EventsCollection(crashes, keys=['NLFID'], beg='COUNTY_LOG_NBR', geom='geometry')

In [63]:
get_columns = ['SPEED_LIMI', 'LANES_NBR', 'LANE_WIDTH']
crashes_ec.df[get_columns] = crashes_ec.merge(roadways_ec)[get_columns].most()

In [64]:
crashes_ec.df

,OBJECTID,DOCUMENT_NBR,CRASH_REPORT_LINK,CRASH_YR,CRASH_MONTH_YEAR,DISTRICT_NBR,NLFID,NLF_JUR_CD,NLF_COUNTY_CD,COUNTY_LOG_NBR,ODOT_LATITUDE_NBR,ODOT_LONGITUDE_NBR,ODOT_FIPS_CD,AREA_CODE,Crash_Date,CRASH_SEVERITY_CD,CRASH_TYPE_CD,DAY_IN_WEEK_CD,DISTRACTED_DRIVER_IND_2,FACILITY_TYPE_CD,FHWA_RDWY_DEPARTURE_IND,FREEWAY_IND,FUNCTIONAL_CLASS_CD,HOUR_OF_CRASH,INCAPAC_INJURIES_NBR,INTERCHANGE_ID,INTERSECTION_ID_CURRENT,INTERSECTION_LEG_ID,INTERSTATE_IND,INV_LOG_NBR,INV_NLFID,IS_COMMERCIAL_RELATED,LIGHT_COND_PRIMARY_CD,LOCAL_REPORT_NUMBER_ID,MAINTENANCE_AUTHORITY_CD,MONTH_OF_CRASH,NHS_CD,NO_INJURY_REPORTED_NBR,NON_INCAPAC_INJURIES_NBR,NUMBER_OF_UNITS_NBR,ODOT_CITY_VILLAGE_TWP_NME,ODOT_CRASH_LOCATION_CD,ODOT_DIR_FROM_REF_CD,ODOT_DIV_UNDIV_IND,ODOT_INTERSECTION_REL_IND,ODOT_LANES_NBR,ODOT_MAINTAINED_HWY_IND,ODOT_MILES_FROM_REF_NBR,ODOT_LOC_ROUTE_PREFIX_CD,ODOT_YOUNG_DRIVER_IND,ODPS_ADDRESS_REFERENCE,ODPS_ALCOHOL_IND,ODPS_CITY_VILLAGE_TWP_NME,ODPS_DRUG_IND,ODPS_LOC_DIR_SUFFIX_CD,ODPS_LOC_IN_WORK_ZONE_CD,ODPS_LOC_ROAD_DIRECTION_CD,ODPS_LOC_ROAD_NME,ODPS_LOC_ROAD_SUFFIX_CD,ODPS_LOC_ROUTE_ID,ODPS_LOC_ROUTE_PREFIX_CD,ODPS_LOC_ROUTE_SUFFIX_CD,ODPS_MILEPOST_REFERENCE,ODPS_MOTORCYCLE_IND,ODPS_REF_DIR_SUFFIX_CD,ODPS_REF_DIRECTION_CD,ODPS_REF_GIVEN,ODPS_REF_POINT_USED_CD,ODPS_REF_ROUTE_ID,ODPS_REF_ROUTE_PREFIX_CD,ODPS_REF_ROUTE_SUFFIX_CD,ODPS_REF_SUFFIX_CD,ODPS_SCHOOL_ZONE_IND,ODPS_SENIOR_DRIVER_IND,ODPS_SPEED_IND,ODPS_TOTAL_FATALITIES_NBR,ODPS_TYPE_OF_WORK_ZONE_CD,ODPS_WORK_ZONE_IND,OPER_ACCESS_CONTROL_CD,POSSIBLE_INJURIES_NBR,ROAD_COND_PRIMARY_CD,ROAD_CONTOUR_CD,SA_RAMP_ID,SA_SEGMENT_ID,SECONDARY_CRASH,STATE_TRUE_LOG_NBR,U1_AGE_NBR,U1_AT_FAULT_IND,U1_CONT_CIR_PRIMARY_CD,U1_DIRECTION_FROM_CD,U1_DIRECTION_TO_CD,U1_DISTRACTED_BY_1_CD,U1_GENDER_CD,U1_IS_ALCOHOL_SUSPECTED,U1_IS_MARIJUANA_SUSPECTED,U1_IS_OTHER_DRUG_SUSPECTED,U1_NON_MOTORIST_LOC_CD,U1_NUMBER_OF_THRU_LANES,U1_OBJECT_STRUCK,U1_ODPS_UNIT_NBR,U1_OPERATINGIN_AUTONOMOUS_MODE,U1_POSTED_SPEED_NBR,U1_PRECRASH_ACTION_CD,U1_SEQ_OF_EVENTS_1_CD,U1_SEQ_OF_EVENTS_2_CD,U1_SEQ_OF_EVENTS_3_CD,U1_SEQ_OF_EVENTS_4_CD,U1_SEQ_OF_EVENTS_5_CD,U1_SEQ_OF_EVENTS_6_CD,U1_SPECIAL_FUNCTION_CD,U1_TRAFFIC_CONTROL_CD,U1_TURN_CD,U1_TYPE_OF_UNIT_CD,U1_UNIT_SPEED_NBR,U2_AGE_NBR,U2_CONT_CIR_PRIMARY_CD,U2_DIRECTION_FROM_CD,U2_DIRECTION_TO_CD,U2_DISTRACTED_BY_1_CD,U2_GENDER_CD,U2_NON_MOTORIST_LOC_CD,U2_ODPS_UNIT_NBR,U2_POSTED_SPEED_NBR,U2_PRECRASH_ACTION_CD,U2_SEQ_OF_EVENTS_1_CD,U2_SEQ_OF_EVENTS_2_CD,U2_SEQ_OF_EVENTS_3_CD,U2_SEQ_OF_EVENTS_4_CD,U2_SEQ_OF_EVENTS_5_CD,U2_SEQ_OF_EVENTS_6_CD,U2_SPECIAL_FUNCTION_CD,U2_TRAFFIC_CONTROL_CD,U2_TURN_CD,U2_TYPE_OF_UNIT_CD,U2_UNIT_SPEED_NBR,U3_CONT_CIR_PRIMARY_CD,U3_ODPS_UNIT_NBR,U3_PRECRASH_ACTION_CD,U3_SEQ_OF_EVENTS_1_CD,U3_SEQ_OF_EVENTS_2_CD,U3_SEQ_OF_EVENTS_3_CD,U3_SEQ_OF_EVENTS_4_CD,U3_SEQ_OF_EVENTS_5_CD,U3_SEQ_OF_EVENTS_6_CD,U3_SPECIAL_FUNCTION_CD,U3_TRAFFIC_CONTROL_CD,U3_TYPE_OF_UNIT_CD,UNRESTRAIN_OCCUPANTS,WEATHER_COND_CD,geometry,SPEED_LIMI,LANES_NBR,LANE_WIDTH
0,192005196,20192005196,https://hsip.dot.state.oh.us/api/report/201920...,2019,2019-01-08,6,CFRACR00025**C,C,FRA,1.117,39.898246,-83.107430,32592,19234.0,01/08/2019,3,6,3,None,NaN,N,N,5.0,12,0,NaN,0a2c9ada-170f-4d61-8d39-7f55305af2c8,63cfe5a2-aa07-44c9-8987-795cf9f3d29b,N,1.117,CFRACR00025**C,N,1,190120,4,1,None,0,2,2,Grove City,2.0,None,N,Y,2.0,N,0.000,CR,N,None,N,Grove City,N,None,0,None,DEMOREST,RD,None,None,None,None,N,None,None,BIG RUN SOUTH,1,None,None,None,RD,None,Y,N,0,0,N,NaN,0,1.0,1,None,13018.0,N,1.117,78.0,Y,3,6,7,1,M,N,N,N,NaN,2,0,1,0,35,1,20,0,0,0,0,0,1,2,NT,1,30,31.0,1.0,5.0,8.0,1.0,M,NaN,2.0,35.0,1.0,20.0,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NT,4.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,POINT (-9251476.789 4851166.664),20,2.0,0
1,192005229,20192005229,https://hsip.dot.state.oh.us/api/report/201920...,2019,2019-01-08,6,SFRAUS00062**C,S,FRA,7.979,39.901006,-83.078072,32592,19234.0,01/08/2019,4,2,3,None,2.0,N,N,3.0,7,0,NaN,1579f981-de91-4bb2-9420-32acffd3f863,3feacac3-694f-44bd-a51f-d8975faa9f0c,N,